<a href="https://colab.research.google.com/github/falloutdey/Atividade2_LLM_e_Estatistica/blob/main/Atividade2_LLM_e_Estatistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>